# 2. 장단기 메모리(Long Short-Term Memory, LSTM)

바닐라 아이스크림이 가장 기본적인 맛을 가진 아이스크림인 것처럼, 앞서 배운 RNN을 가장 단순한 형태의 RNN이라고 하여 바닐라 RNN(Vanilla RNN)이라고 합니다. (케라스에서는 SimpleRNN) 바닐라 RNN 이후 바닐라 RNN의 한계를 극복하기 위한 다양한 RNN의 변형이 나왔습니다. 이번 챕터에서 배우게 될 LSTM도 그 중 하나입니다. 앞으로의 설명에서 LSTM과 비교하여 RNN을 언급하는 것은 전부 바닐라 RNN을 말합니다.

## 1) 바닐라 RNN의 한계

<img src=lstm_image1_ver2.png width=300>

- 바닐라 RNN은 출력 결과가 이전의 계산 결과에 의존한다고 했었음
- **BUT** 바닐라 RNN은 비교적 짧은 시퀀스(sequence)에 대해서만 효과를 보이는 단점 존재
    - 바닐라 RNN의 시점(time step)이 길어질수록 앞의 정보가 뒤로 충분히 전달되지 못하는 현상이 발생
- 위의 그림은 첫번째 입력값인 x1의 정보량을 짙은 남색으로 표현했을 때, 색이 점차 얕아지는 것으로 시점이 지날수록 x1의 정보량이 손실되어가는 과정을 표현
- 가장 중요한 정보가 시점의 앞 쪽에 있을 경우도 있어서 손실을 줄여아 할 수도 있음
    - 예를 들어, "모스크바에 여행을 왔는데 건물도 예쁘고 먹을 것도 맛있었어. 그런데 글쎄 직장 상사한테 전화가 왔어. 어디냐고 묻더라구 그래서 나는 말했지. 저 여행왔는데요. 여기 ____"
    
    
This is **장기 의존성 문제(the problem of Long-Term Dependencies)**

## 2) 바닐라 RNN 내부 열어보기

<img src=vanilla_rnn_ver2.png width=400>

$h_t = tanh(W_xx_t + W_hh_{t-1} + b)$  

- 바닐라 RNN은 $x_t$와 $h_{t−1}$이라는 두 개의 입력이 각각의 가중치와 곱해져서 메모리 셀의 입력이 됨.
- 다시 이것을 $tanh$의 입력으로 사용하면 은닉층의 출력인 은닉상태(hidden state)가 됨.

## 3) LSTM(Long Short-Term Memory)

<img src=vaniila_rnn_and_different_lstm_ver2.png width=500>

- 전통적인 RNN의 이러한 단점을 보완한 RNN의 일종을 **장단기 메모리(Long Short-Term Memory)**라고 하며, 줄여서 **LSTM**이라고 함
- LSTM은 은닉층의 메모리 셀에 입력 게이트, 망각 게이트, 출력 게이트를 추가하여 불필요한 기억을 지우고, 기억해야할 것들을 정함
- 즉, LSTM은 은닉 상태(hidden state)를 계산하는 식이 전통적인 RNN보다 조금 더 복잡해졌으며 셀 상태(cell state)라는 값을 추가
- 위의 그림에서  t시점의 셀 상태를 Ct로 표현
- LSTM은 RNN과 비교하여 긴 시퀀스의 입력을 처리하는데 탁월한 성능을 보임

<img src=cellstate.png width=400>

- **셀 상태** : 위의 그림에서 왼쪽에서 오른쪽으로 가는 굵은 선
- 셀 상태 또한 이전에 배운 은닉 상태처럼 이전 시점의 셀 상태가 다음 시점의 셀 상태를 구하기 위한 입력으로서 사용
- 은닉상태값과 셀 상태값을 구하기 위해 3개의 추가 게이트를 사용(삭제 게이트, 입력 게이트, 출력 게이트) ----> 이 3개의 게이트에는 공통적으로 시그모이드 함수 존재
- 시그모이드 함수를 지나면 0과 1사이의 값이 나오는데 이 값을 가지고 게이트를 조절
    - 이하 식에서 $\sigma$는 시그모이드 함수를 의미
    - $W_{xi}, W_{xg}, W_{xf}, W_{xo}$는 $x_t$와 함께 각 게이트에서 사용되는 4개의 가중치
    - $W_{hi}, W_{hg}, W_{hf}, W_{ho}$는 $h_{t-1}$과 함께 각 게이트에서 사용되는 4개의 가중치
    - $b_i, b_g, b_f, b_o$는 각 게이트에서 사용되는 4개의 편향

### (1) 입력 게이트

<img src=inputgate.png width=400>

$i_t = \sigma(W_{xi}x_t + W_{hi}h_{t-1} + b_i)$  
$g_t = tanh(W_{xg}x_t + W_{hg}h_{t-1} + b_g)$

- 입력 게이트는 현재 정보를 기억하기 위한 게이트
- $i_t, g_t$ 두 개의 값을 가지고 기억할 정보의 양을 정함(이건 아래에서 보여줄 예정)

### (2) 삭제 게이트

<img src=forgetgate.png width=400>

$f_t = \sigma(W_{xf}x_t + W_{hf}h_{t-1} + b_f)$

- 삭제 게이트는 기억을 삭제하기 위한 게이트
- 0에 가까울 수록 정보가 많이 삭제된 것, 1에 가까울 수록 정보를 온전히 기억

### (3) 셀 상태(장기 상태)

<img src=cellstate2.png width=400>

$C_t = f_t \circ C_{t-1} + i_t \circ g_t $

- 셀 상태 $C_t$를 LSTM에서는 장기 상태로 부르기도 함
- 여기서 $\circ$는 entrywise product를 의미

입력 게이트에서 선택된 기억을 삭제 게이트의 결과값과 더합니다. 이 값을 현재 시점 t의 셀 상태라고 하며, 이 값은 다음 t+1 시점의 LSTM 셀로 넘겨집니다.

삭제 게이트와 입력 게이트의 영향력을 이해해봅시다. 만약 삭제 게이트의 출력값인 ft가 0이 된다면, 이전 시점의 셀 상태값인 Ct−1은 현재 시점의 셀 상태값을 결정하기 위한 영향력이 0이 되면서, 오직 입력 게이트의 결과만이 현재 시점의 셀 상태값 Ct을 결정할 수 있습니다. 이는 삭제 게이트가 완전히 닫히고 입력 게이트를 연 상태를 의미합니다. 반대로 입력 게이트의 it값을 0이라고 한다면, 현재 시점의 셀 상태값 Ct는 오직 이전 시점의 셀 상태값 Ct−1의 값에만 의존합니다. 이는 입력 게이트를 완전히 닫고 삭제 게이트만을 연 상태를 의미합니다. 결과적으로 삭제 게이트는 이전 시점의 입력을 얼마나 반영할지를 의미하고, 입력 게이트는 현재 시점의 입력을 얼마나 반영할지를 결정합니다.

### (4) 출력 게이트와 은닉 상태(단기 상태)

<img src=outputgateandhiddenstate.png width=400>

$o_t = \sigma(W_{xo}x_t + W_{ho}h_{t-1} + b_o)$  
$h_t = o_t \circ tanh(C_t)$

- 출력 게이트는 현재 시점 t의 x값과 이전 시점 t-1의 은닉 상태가 시그모이드 함수를 지난 값
- 해당 값은 현재 시점 t의 은닉 상태를 결정하는 일에 쓰이게 됨